In [1]:
# Importing libraries to work with
from astropy.io.votable import parse_single_table as pst
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
def opt_filter(non_matched_tab):
    """Removes SDSS detections that have Petrosian r-mag > 22.5 since errors associated with the
       Petrosian radius below this is stable
       
    non_matched_tab = table of optical data containing Petrosian r-band magnitude 
    
    Returns only those objects that meet the aforementioned criteria
    """
    
    tab_rp_valid = (non_matched_tab.loc[non_matched_tab.petromag_r < 22.5]).values.tolist()
    
    return tab_rp_valid

In [3]:
def xray_filter(tab_vot):
    """ Filters all the CSCview cross-matched sources based on the following conditions for x-ray data:
        1. Detections must be above 3-sigma significance to remove spurious detections
        2. objects with Major_axis of energy bands < 1 indicating observation as a point source in the given band
        To check the energy ranges of each band, visit http://cxc.harvard.edu/csc/columns/ebands.html
    
    tab_vot = table returned by CSCview with cross-matched X-ray targets
    
    Returns only those objects that meet the aforementioned criteria
    """
    
    tab_2CXO = tab_vot.array[(tab_vot.array['master_source.significance']>3)
                             & (tab_vot.array['master_source.major_axis_m']<1)
                             & (tab_vot.array['master_source.major_axis_b']<1) 
                             & (tab_vot.array['master_source.major_axis_h']<1) 
                             & (tab_vot.array['master_source.major_axis_s']<1) 
                             & (tab_vot.array['master_source.major_axis_u']<1)].tolist()
    return tab_2CXO

In [4]:
def final_match(tab_xray, tab_opt):
    """ Appends SDSS and cross-matched tables
    
    tab_xray = filtered table of X-ray targets
    tab_opt = filtered table of optical data
    
    Returns table after matching filtered X-ray and optical sources only consisting of parameters of interest
    """
    
    tab_match_final = []    # Initiating list to be made into final table
    c=0    # Initiating counter to see if there are any mathced sources after filtration 
    for xray in tab_xray:
        for opt in tab_opt:
            if xray[0] == opt[0]:    # matching tables based on common entity: SDSS SpecObjID
                tot = [opt[0],opt[1],opt[2],opt[3],opt[4],opt[5],opt[6],
                       opt[134],opt[140],opt[152],opt[232],opt[233],opt[234],
                       opt[235],opt[236],opt[237],opt[238],opt[239],opt[240],
                       opt[241],opt[242],opt[243],opt[244],opt[245],opt[246],
                       opt[247],opt[248],opt[249],opt[250],opt[251],opt[252],
                       opt[253],opt[254],opt[255],opt[256],opt[257],opt[258],
                       opt[261],opt[262],
                       xray[3],xray[4],xray[5],xray[6],xray[7],xray[9],
                            xray[20],xray[30],xray[40],xray[50],xray[60]]
                tab_match_final.append(tot)
                c=1
    if c==0:
            print("No match")
    
    return tab_match_final

In [5]:
def isoff(ra_x, ra_o, dec_x, dec_o, raerr_x, raerr_o, decerr_x, decerr_o, rp):
    
    """ Calculates the offset between the X-ray and optical source coordinates and checks if it is >= 5 * r_p
    
    ra_x, dec_x = X-ray source coordinates
    dec_o, dec_x = Optical source coordinates
    raerr_x, decerr_x = Errors associated with X-ray coordinates
    raerr_o, decerr_o = Errors associated with Optical coordinates
    rp = Petrosian radius
    
    Returns the offset in terms of arcsec and r_p, the error associated with the offset and flag for above criteria
    """
    
    # calculating the offset
    ra_diff = abs(ra_x - ra_o)
    dec_diff = abs(dec_x - dec_o)
    offset_arcsec = np.sqrt(ra_diff**2 + dec_diff**2)
    offset_rp = offset_arcsec/(rp)
    
    # calculating the error in offset
    del_ra_diff = np.sqrt(raerr_x**2 + raerr_o**2)
    del_dec_diff = np.sqrt(decerr_x**2 + decerr_o**2)
    del_offset = np.sqrt((ra_diff/offset_arcsec * del_ra_diff)**2 + (dec_diff/offset_arcsec * del_dec_diff)**2)
    
    # checking if the offset is above 5 * r_p
    if offset_arcsec/del_offset >= 5:
        result = 'True'
    else:
        result = 'False'   
    
    return offset_arcsec, offset_rp, del_offset, result

In [6]:
def offset_det(tab_cand):
    """ Checks if the offset between optical and X-ray sources are above 5 * r_p and discards objects if not 
    
    tab_cand = filtered cross-matched table of optical and x-ray data
    
    Returns those objects that meets the above criteria
    """
    
    tab_final = []    # Initializing objects that 
    for obj in tab_cand:
        obj_copy = obj.copy()
        offset_arcsec, offset_rp, del_offset, result = isoff(obj_copy[40]*3600,obj_copy[3]*3600,obj_copy[41]*3600,
                                                             obj_copy[4]*3600, obj_copy[42],obj_copy[37],
                                                             obj_copy[43],obj_copy[38], obj_copy[14])
        obj_copy.append(offset_arcsec)
        obj_copy.append(offset_rp)
        obj_copy.append(del_offset)
        obj_copy.append(result)
        if result == 'True':
            tab_final.append(obj_copy)
    return(tab_final)

In [7]:
# Reading SDSS data prior to cross-match performed by CSCview. The rowindex count starts from 0 when using pandas
non_matched_mergs = pd.read_csv(r"hd_merg_sat_dc1408.csv")
non_matched_irreg = pd.read_csv(r"hd_irreg_sat_dc1408.csv")
non_matched_distr = pd.read_csv(r"hd_dist_sat_dc1408.csv")

# Reading 2CXO data after cross-matching against SDSS GZ2 data. 
mergs_vot = pst("cscresults_hd_merg_sat.vot")
irreg_vot = pst("cscresults_hd_irreg_sat.vot")
distr_vot = pst("cscresults_hd_dist_sat.vot")

In [8]:
# Filtering optical data
merg_opt = opt_filter(non_matched_mergs)
irreg_opt = opt_filter(non_matched_irreg)
dist_opt = opt_filter(non_matched_distr)

In [9]:
# Filtering X-ray data
merg_xray = xray_filter(mergs_vot)
irreg_xray = xray_filter(irreg_vot)
dist_xray = xray_filter(distr_vot)

C:\Users\devar\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in less


In [10]:
# Merging filtered optical and X-ray data
merg_cand = final_match(merg_xray, merg_opt)
irreg_cand = final_match(irreg_xray, irreg_opt)
dist_cand = final_match(dist_xray, dist_opt)

In [11]:
# Offset between optical and x-ray sources determined and sources filtered accordingly
mergs = offset_det(merg_cand)
irreg = offset_det(irreg_cand)
distr = offset_det(dist_cand)

In [12]:
# Saving filtered data to csv files based on their morphological classification
with open('merg_cand_1.csv', 'w', newline='') as fp:
    a = csv.writer(fp, delimiter=',')
    a.writerows(mergs)

with open('irreg_cand_1.csv', 'w', newline='') as fp:
    b = csv.writer(fp, delimiter=',')
    b.writerows(irreg)

with open('dist_cand_1.csv', 'w', newline='') as fp:
    c = csv.writer(fp, delimiter=',')
    c.writerows(distr)